In [5]:
from model import BinaryClassificationModel, Classifier

import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,Subset,random_split
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [6]:
seed = 0
random.seed(seed)
np.random.seed(seed)

## Data

In [7]:
data = pd.read_csv('./data.csv')

data_x = data.iloc[:,:-1]
data_y = data.iloc[:,-1]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# 데이터 표준화
standardized_data_x = scaler.fit_transform(data_x)
data_s_x = pd.DataFrame(standardized_data_x, columns = data_x.columns)

X_train, X_test, y_train, y_test = train_test_split(data_s_x, data_y, test_size=0.2, random_state=0)

In [27]:
# Tensor로 변환
X_train, X_test, y_train, y_test = train_test_split(standardized_data_x, data_y, test_size=0.2, random_state=0)

X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
y_train_tensor = torch.tensor(y_train.to_list(),dtype = torch.float32).squeeze(-1)  # y를 (1000, 1) 크기로 변환

X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
y_test_tensor = torch.tensor(y_test.to_list(),dtype = torch.float32).squeeze(-1)  # y를 (1000, 1) 크기로 변환

In [18]:
class SingleHeadSelfAttention(nn.Module):
    def __init__(self, embed_size):
        super(SingleHeadSelfAttention, self).__init__()
        self.embed_size = embed_size

        self.values = nn.Linear(embed_size, embed_size, bias=False)
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(query)

        # Scaled dot-product attention
        energy = torch.einsum("qd,kd->qk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** 0.5), dim=1)
        out = torch.einsum("qk,vd->qd", [attention, values])
        out = self.fc_out(out)
        return out, attention

In [34]:
class MachinePredictionModel(nn.Module):
    def __init__(self, embed_size):
        super(MachinePredictionModel, self).__init__()
        self.embed_size = embed_size
        self.num_sensors = num_sensors
        self.num_machines = num_machines

        self.sensor_embedding = nn.Linear(num_machines, embed_size)
        self.machine_embedding = nn.Linear(num_sensors, num_sensors)
        self.attention = SingleHeadSelfAttention(embed_size)
        self.fc_out = nn.Linear(embed_size, 1)  # For binary classification

    def forward(self, x, mask=None):
        num_machines, num_sensors = x.shape
        assert num_machines == self.num_machines
        assert num_sensors == self.num_sensors
        
        # Apply sensor embedding to each machine separately
        machine_emb = self.machine_embedding(x) # 순서 바뀌면 안됨 원본 > machine, 원본 > layer 통과
        x = self.sensor_embedding(x.T)  # Shape: (num_machines, embed_size)
 
        
        # Apply attention within each machine separately
        # x = x.view(num_machines, self.num_sensors, self.embed_size)  # Shape: (num_machines, num_sensors, embed_size)
        sensor_att, attention = self.attention(x, x, x, mask)  
        x = torch.mm(machine_emb, sensor_att) # Shape: (num_machines, embed_size)

        # Final output layer for each machine
        out = self.fc_out(x).squeeze(-1)  # Shape: (num_machines)
        return out

In [36]:
# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed_size = 32

model = MachinePredictionModel(
    embed_size=embed_size
).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()  # Assuming binary classification

In [37]:
# 학습 루프
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    running_loss += loss.item() * inputs.size(0)

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')

# 검증 루프
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    outputs = model(X_test_tensor)
    loss = criterion(outputs, y_test_tensor)
    val_loss += loss.item() * X_test_tensor.size(0)
    predicted = (outputs > 0.5).float()
    total += y_test_tensor.size(0)
    correct += (predicted == y_test_tensor).sum().item()

accuracy = correct / total
print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')
fold_results.append((val_loss, accuracy))

# 교차 검증 결과 출력
fold_results = np.array(fold_results)
mean_loss = fold_results[:, 0].mean()
mean_accuracy = fold_results[:, 1].mean()
print(f'5-Fold Cross-Validation Results:')
print(f'Average Validation Loss: {mean_loss:.4f}')
print(f'Average Validation Accuracy: {mean_accuracy:.4f}')

Epoch 1/100, Loss: 0.5815
Epoch 2/100, Loss: 0.8088
Epoch 3/100, Loss: 0.6187
Epoch 4/100, Loss: 0.5833
Epoch 5/100, Loss: 0.6460
Epoch 6/100, Loss: 0.5619
Epoch 7/100, Loss: 0.5380
Epoch 8/100, Loss: 0.5510
Epoch 9/100, Loss: 0.5567
Epoch 10/100, Loss: 0.5458
Epoch 11/100, Loss: 0.5258
Epoch 12/100, Loss: 0.5076
Epoch 13/100, Loss: 0.4988
Epoch 14/100, Loss: 0.4994
Epoch 15/100, Loss: 0.4998
Epoch 16/100, Loss: 0.4920
Epoch 17/100, Loss: 0.4791
Epoch 18/100, Loss: 0.4671
Epoch 19/100, Loss: 0.4586
Epoch 20/100, Loss: 0.4529
Epoch 21/100, Loss: 0.4481
Epoch 22/100, Loss: 0.4428
Epoch 23/100, Loss: 0.4365
Epoch 24/100, Loss: 0.4292
Epoch 25/100, Loss: 0.4215
Epoch 26/100, Loss: 0.4142
Epoch 27/100, Loss: 0.4077
Epoch 28/100, Loss: 0.4021
Epoch 29/100, Loss: 0.3971
Epoch 30/100, Loss: 0.3921
Epoch 31/100, Loss: 0.3865
Epoch 32/100, Loss: 0.3806
Epoch 33/100, Loss: 0.3746
Epoch 34/100, Loss: 0.3691
Epoch 35/100, Loss: 0.3641
Epoch 36/100, Loss: 0.3597
Epoch 37/100, Loss: 0.3555
Epoch 38/1

AssertionError: 